In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
import networkx as nx
import scipy.spatial.distance

In [2]:
df = pd.read_csv('Full_data.csv',encoding='cp1252')

In [3]:
df.head(10)

,id,skills,titles,major
0,/in/cassandra-vose/,"['Data Warehousing', 'Data Analysis', 'Busines...","['Senior Manager/Data Engineer', 'Consultant']","['Bachelor of Science, Civil & Environmental E..."
1,/in/samitra-harrington-3ab715141/,1,['Research Ambassador'],"[""Bachelor's degree, Sociology""]"
2,/in/kavyadagli/,"['Python', 'Java', 'JavaScript', 'C', 'HTML', ...","['Software Engineer Intern', 'Teaching Assista...","['Bachelor of Science - BS, Computer Science']"
3,/in/melvin-molina/,"['Java', 'C (Programming Language)', 'Python (...","['Associate Software Engineer', 'High School T...","['Bachelor of Science - BS, Computer Science',..."
4,/in/sarah-nguyen-4706b4205/,['NaN'],['NaN'],"['Bachelor of Science - BS, Community Health a..."
5,/in/shivppatel/,"['Java', 'Python', 'JavaScript', 'Git', 'Probl...","['University of Maryland', 'University of Mary...",['Bachelor of Science - BS']
6,/in/excela/,"['Leadership', 'Research', 'Microsoft Office',...","['Software Automation Engineer (Intern)', 'Und...","['Bachelor of Engineering - BE, Computer Engin..."
7,/in/adam-jacques/,"['SolidWorks', 'Python (Programming Language)'...","['University Recreation and Wellness', 'Univer...","['Bachelor of Science - BS, Mechanical Enginee..."
8,/in/azariah-knox-529a95170/,['NaN'],['NaN'],"['Bachelor of Science - BS, Mechanical Enginee..."
9,/in/clydeapaid/,"['Microsoft Office', 'Financial Analysis', 'Ma...","['Manager', 'Pathways Operations Management In...","['Master of Business Administration (M.B.A.)',..."


In [4]:
#Remove the rows that empty rows
df = df[df['skills'] != "['NaN']"]	
df = df[df['titles'] != "['NaN']"]	
df = df[df['major'] != "['NaN']"]	
#Remove the rows that empty rows
df = df[df.skills.apply(lambda x: not(x.isnumeric()))]
df = df[df.titles.apply(lambda x: not(x.isnumeric()))]
df = df[df.major.apply(lambda x: not(x.isnumeric()))]
#Converts string version of lists into lists 
df['skills'] = df.skills.apply(lambda x: literal_eval(str(x)))
df['titles'] = df.titles.apply(lambda x: literal_eval(str(x)))
df['major'] = df.major.apply(lambda x: literal_eval(str(x)))
#Resets row counter
df = df.reset_index(drop=True)

print(type(df['major'][0]))
print(type(df['titles'][0]))
print(type(df['skills'][0]))


<class 'list'>
<class 'list'>
<class 'list'>


In [5]:
total = []
indexcount = []
for index, row in df.iterrows():
    cur_title = []
    for title in row['titles']:
        cur_title.append(title.rsplit('<span class')[0])
    cur_title = list(set(cur_title))
    df.loc[index]['titles'] = cur_title
    cur_major = ''
    for major in row['major']:
        string_major = major.rsplit('<span class')[0]
        if 'Bachelor' in string_major:
            cur_major = string_major 
            continue
    if not cur_major:
        indexcount.append(index)
    df.loc[index]['major'] = cur_major
    cur_skill = []
    for skill in row['skills']:
        cur_skill.append(skill.rsplit('<span class')[0])
    cur_skill = list(set(cur_skill))
    df.loc[index]['skills'] = cur_skill
print(indexcount)
df = df.drop(indexcount)
df = df.reset_index(drop=True)

[15, 22, 23, 24, 29, 31, 33, 36, 48, 55, 60, 62, 73, 80, 83, 85, 89, 92, 94, 97, 99, 108, 111, 115, 119, 126, 127, 128]


In [6]:
df_origin = df.copy()
df.head()

,id,skills,titles,major
0,/in/cassandra-vose/,"[Leadership, Data Analysis, SQL, Microsoft Off...","[Consultant, Senior Manager/Data Engineer]","Bachelor of Science, Civil & Environmental Eng..."
1,/in/kavyadagli/,"[Java, C, JavaScript, HTML, Python]",[Teaching Assistant (Object-Oriented Programmi...,"Bachelor of Science - BS, Computer Science"
2,/in/melvin-molina/,"[Java, Python (Programming Language), C (Progr...","[Technical Researcher, Computer Science Tutor,...","Bachelor of Science - BS, Computer Science"
3,/in/shivppatel/,"[Java, Problem Solving, JavaScript, Git, Commu...",[University of Maryland],Bachelor of Science - BS
4,/in/excela/,"[Leadership, Tutoring, English, Research, Prob...","[Undergraduate Student Researcher, Engagement ...","Bachelor of Engineering - BE, Computer Enginee..."


In [7]:
total = []
for index, row in df.iterrows():
    for skill in row['skills']:
        total.append(skill)
#print(len(total))
print('Number of different skills:',len(list(set(total))))
unique_skills = list(set(total))


Number of different skills: 522


In [8]:
unique_majors = list(set(df['major'].values.tolist()))
len(unique_majors)

89

Graph

In [9]:
major_graph = nx.Graph() 
for major in unique_majors:
    for index, row in df.iterrows():
        if row['major'] == major:
            for cur_title in row['titles']:
                major_graph.add_edge(major,cur_title)
nx.write_gexf(major_graph, "majors.gexf")                
                

In [10]:
skill_graph = nx.Graph() 
for uniskil in unique_skills:
    for index, row in df.iterrows():
        if uniskil in row['skills']:
            for job in row['titles']:
                skill_graph.add_edge(uniskil,job)
nx.write_gexf(skill_graph, "jobs.gexf")         

Similarity Analysis


In [11]:

skill_df = pd.DataFrame
mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('skills')),
                index=df.index,
                columns=mlb.classes_))

In [12]:
df.shape

(112, 525)

In [13]:
df.head()

,id,titles,major,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
0,/in/cassandra-vose/,"[Consultant, Senior Manager/Data Engineer]","Bachelor of Science, Civil & Environmental Eng...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,/in/kavyadagli/,[Teaching Assistant (Object-Oriented Programmi...,"Bachelor of Science - BS, Computer Science",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,/in/melvin-molina/,"[Technical Researcher, Computer Science Tutor,...","Bachelor of Science - BS, Computer Science",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,/in/shivppatel/,[University of Maryland],Bachelor of Science - BS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,/in/excela/,"[Undergraduate Student Researcher, Engagement ...","Bachelor of Engineering - BE, Computer Enginee...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data = df.iloc[:,3:,]
data.head()

,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,Adobe Creative Suite,Adobe Illustrator,Adobe Indesign CC,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
arr = np.asarray(data)
arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [16]:
target = df[df['id'] == '/in/kavyadagli/'].iloc[:,3:,]
target = np.asarray(target)
print(target)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [17]:
distances = scipy.spatial.distance.cdist(arr,target,'cosine')[:,0]

query_distances = list(zip(df.id, distances))
print(query_distances)

[('/in/cassandra-vose/', 1.0), ('/in/kavyadagli/', 2.220446049250313e-16), ('/in/melvin-molina/', 0.7418011102528389), ('/in/shivppatel/', 0.4522774424948339), ('/in/excela/', 1.0), ('/in/adam-jacques/', 1.0), ('/in/clydeapaid/', 1.0), ('/in/asiya-sarbasova-pmp-25a03322/', 1.0), ('/in/connor-guidera/', 1.0), ('/in/adeolaawowale/', 1.0), ('/in/zayd-choudhury/', 0.8), ('/in/joy-j-wang/', 1.0), ('/in/neha-wadhwani-360949143/', 1.0), ('/in/nekesiha-hall/', 1.0), ('/in/akash-deep-umd/', 0.8709005551264195), ('/in/cyjean/', 0.49290744716289014), ('/in/marlabar/', 1.0), ('/in/rishipatel94/', 1.0), ('/in/devanshi-shah-21592713a/', 1.0), ('/in/nicole-vertescher-17519a129/', 1.0), ('/in/linda-johnson-mba-5a2a2a64/', 1.0), ('/in/adamhoward56/', 0.7418011102528389), ('/in/helawe/', 1.0), ('/in/jeremy-hess-41524016a/', 0.850928801500014), ('/in/hannah-allison-9853b836/', 1.0), ('/in/timisanni/', 0.7763932022500211), ('/in/nygel-taylor-35096494/', 1.0), ('/in/zain-zaidi-02a027168/', 1.0), ('/in/sani

In [18]:
sorted_by_second = sorted(query_distances, key=lambda tup: tup[1])
sorted_by_second

[('/in/kavyadagli/', 2.220446049250313e-16),
 ('/in/marvinxu/', 0.32580013753675796),
 ('/in/shivppatel/', 0.4522774424948339),
 ('/in/cyjean/', 0.49290744716289014),
 ('/in/namanmolri/', 0.49290744716289014),
 ('/in/saniyanazeer/', 0.5527864045000421),
 ('/in/jgao1238/', 0.6348516283298893),
 ('/in/kahadze/', 0.6348516283298893),
 ('/in/william-l-ju/', 0.683772233983162),
 ('/in/michael-wittner-559962101/', 0.683772233983162),
 ('/in/rohit-mishra-9a9536229/', 0.683772233983162),
 ('/in/ali-p-82688713b/', 0.683772233983162),
 ('/in/hana-huie-8902671b3/', 0.683772233983162),
 ('/in/akhilbothara/', 0.7000000000000001),
 ('/in/mmosazgh/', 0.7139612232263224),
 ('/in/shane-khant-soe-moe-24ab45113/', 0.7303200550147031),
 ('/in/moreno-aracely/', 0.7303200550147031),
 ('/in/melvin-molina/', 0.7418011102528389),
 ('/in/adamhoward56/', 0.7418011102528389),
 ('/in/sruthi-rajarajan/', 0.7418011102528389),
 ('/in/agarwalseema/', 0.7418011102528389),
 ('/in/timisanni/', 0.7763932022500211),
 ('/in

In [19]:
found = df[df['id']=='/in/linda-johnson-mba-5a2a2a64/']
original = df[df['id'] == '/in/kavyadagli/']
left = []
right = []
for i in df.columns:
    if found[i].values[0] == 1:
        left.append(i)
    if original[i].values[0] == 1:
        right.append(i)
mid = set(left).union(set(right))
print(len(mid))
print(mid)


39
{'Leadership', 'WMS Implementations', 'Microsoft Word', 'English', 'Lean Six Sigma', 'Manufacturing', 'PowerPoint', 'Mechanical Engineering', 'Event Planning', 'Social Media', 'CATIA', 'C', 'Java', 'Customer Service', 'PTS', 'WMS (Warehouse Management System)', 'SolidWorks', 'Oracle Database', 'Project Management', 'Research', 'Windows', 'Kaizen Leadership', 'Matlab', 'Data Analysis', 'Microsoft Office', 'Cross-functional Team Leadership', 'Python', 'Microsoft Excel', 'Microsoft Project', 'Team Leadership', 'Time Management', 'Problem Solving', 'JavaScript', 'Public Speaking', 'Microsoft PowerPoint', 'HTML', 'Management', 'Engineering', 'Teamwork'}


In [20]:
found[i].values[0]

0

In [21]:
found

,id,titles,major,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
20,/in/linda-johnson-mba-5a2a2a64/,[Intern for Cellulose Esters and Specialty Pla...,"Bachelor of Science (BS), Mechanical Engineering",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
original

,id,titles,major,3D Computer Aided Design (3D CAD),3D Printing,3D Prototyping,Accessibility,Account Management,Account Reconciliation,Accounting,...,XML,YouTube,behavior planning,iBwave,jQuery,mission planning,path planning,perception,python,trajectory planning
1,/in/kavyadagli/,[Teaching Assistant (Object-Oriented Programmi...,"Bachelor of Science - BS, Computer Science",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_origin[df_origin['id'] == '/in/linda-johnson-mba-5a2a2a64/']['titles'].values


array([list(['Intern for Cellulose Esters and Specialty Plastics Department', 'How Stuff Works Teaching Assistant', 'Undergraduate Research Assistant', 'Manufacturing Operations Supervisor', 'Corporate Relations Board Chair', 'Conference Planning Chair', 'Mechanical Engineering Intern, Global Engineering Service Center', 'NPI OLM Program Manager', 'Associate Supply Chain Engineer', 'Mechanical Engineering Intern', 'Program Manager'])],
      dtype=object)